In [113]:
import numpy as np
import pandas as pd

In [114]:
#temporada: verano, 1 es verano, 0 es invierno.
inv_verano = pd.read_excel('../02_data/verano.xlsx', sheet_name='veranos', skiprows=1)
inv_verano = pd.melt(inv_verano, id_vars=['zona'], value_vars=inv_verano.columns[1:] , var_name='mes_no', value_name='temporada')

In [115]:
import datetime
import dateutil.relativedelta as relativedelta

mes_inicio = 9
año_inicio = 2023

date_inicio = datetime.datetime(year=año_inicio, month=mes_inicio, day=10)
primer_periodo = str(date_inicio.year) + '-' + str(date_inicio.month).zfill(2)

periodos = []
horizonte_planificacion = 12
for mes in range(0, horizonte_planificacion + 1):
    aux_per = date_inicio + relativedelta.relativedelta(months=mes)
    periodos.append(str(aux_per.year) + '-' + str(aux_per.month).zfill(2))

periodos = pd.DataFrame(periodos, columns=['mes_real'])
periodos['año'] = periodos['mes_real'].str[:4]
periodos['mes_no'] = periodos['mes_real'].str[5:7].astype(int)
periodos['mes_planificado'] = periodos.index
periodos = pd.merge(periodos, inv_verano, how='left', on='mes_no')
periodos.head()

,mes_real,año,mes_no,mes_planificado,zona,temporada
0,2023-09,2023,9,0,chillan,0
1,2023-09,2023,9,0,constitucion,0
2,2023-09,2023,9,0,arauco,0
3,2023-09,2023,9,0,valdivia,0
4,2023-10,2023,10,1,chillan,1


In [116]:
rango_mes_tipo = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='rango_mes_tipo')
rangos_demanda = pd.read_excel('../02_data/parametro_antiguedad.xlsx', sheet_name='parametro_antiguedad')

rangos_demanda_ext = pd.DataFrame(columns=['id_tipo_rango', 'rango', 'antiguedad'])
for ix in rangos_demanda.index:
    for ant in range(rangos_demanda.at[ix, 'inicio_rango'], rangos_demanda.at[ix, 'fin_rango'] + 1):
        aux = [[rangos_demanda.at[ix, 'id_tipo_rango'], rangos_demanda.at[ix, 'rango'], ant ]]
        aux = pd.DataFrame(aux, columns=['id_tipo_rango', 'rango', 'antiguedad'])
        rangos_demanda_ext = pd.concat([rangos_demanda_ext, aux], ignore_index=True)

rangos_demanda_ext['antiguedad'] = rangos_demanda_ext['antiguedad'].astype(np.int64).astype(str)
rangos_demanda_ext.head()

,id_tipo_rango,rango,antiguedad
0,1,0-3,0
1,1,0-3,1
2,1,0-3,2
3,1,0-3,3
4,1,4-6,4


In [117]:
#Lectura de parametros planta
parametros_planta = pd.read_excel('../02_data/plantas.xlsx', sheet_name='parametros_planta')
parametros_planta['stock_ideal_planta'] = parametros_planta['ideal_dias_autonomia'] * parametros_planta['consumo_diario'] 

#Lectura consumo plantas
consumo_plantas = pd.read_excel('../02_data/plantas.xlsx', sheet_name='consumo_plantas')
consumo_plantas = pd.melt(consumo_plantas, consumo_plantas.columns[:2], consumo_plantas.columns[2:], var_name='mes_planificado', value_name='volumen_consumo')

## Matriz de demandas.

#Consumo_planta_tipo + parametros_planta_tipo
#Calculo demanda
df = pd.merge(consumo_plantas, parametros_planta, on=['nombre_planta', 'producto'])
df.loc[df['mes_planificado'] != 1, 'stock_inicial'] = 0
df['demanda'] = df['volumen_consumo'] + df['stock_ideal_planta'] - df['stock_inicial']

#Agregar los tipos de rango por mes
df = pd.merge(df, rango_mes_tipo, how='left', on=['nombre_planta', 'producto', 'mes_planificado'])
df = pd.merge(df, rangos_demanda, how='left', on='id_tipo_rango')
df['demanda_rango'] = df['demanda'] * df['porcentaje']
df = df[['nombre_planta', 'producto', 'mes_planificado', 'id_tipo_rango', 'rango', 'inicio_rango', 'fin_rango', 'demanda_rango']].dropna()
df['id_tipo_rango'] = df['id_tipo_rango'].astype(int)
df['inicio_rango'] = df['inicio_rango'].astype(int)
df['fin_rango'] = df['fin_rango'].astype(int)

demanda_plantas = df.copy()
demanda_plantas.to_csv('../03_pydata/demanda_plantas.csv', index=False, sep=';', decimal=',')

In [118]:
#Carga Ingresos: Compras y produccion
base_ingresos = pd.read_excel('../02_data/ingresos.xlsx', sheet_name='ingresos')
display(base_ingresos.head())

#Carga Ingreso: Volumen mensual
vol_ingresos = pd.read_excel('../02_data/ingresos_volumen.xlsx', sheet_name='ingresovolumen')
vol_ingresos = pd.merge(base_ingresos, vol_ingresos, on='id_ingreso', how='left')

vol_ingresos.to_csv('../03_pydata/ingresos_volumen.csv', sep=';', decimal=',', index=False)

,id_ingreso,nombre_ingreso,nombre_planta,producto,zona,costo existencia,costo transporte,tipo_ingreso,almacen_verano,almacen_invierno,tipo_almacen,cosecha_compra
0,1,produccion EUNI SC constitucion,planta1,EUNI SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
1,2,produccion EUNI arauco sc,planta1,EUNI SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha
2,3,produccion EUNI valdivia sc,planta1,EUNI SC,valdivia,10,20,directo_planta,N_A,N_A,bosque,cosecha
3,4,produccion EUGL constitucion sc,planta1,EUGL SC,constitucion,10,20,directo_planta,N_A,N_A,bosque,cosecha
4,5,produccion EUGL arauco sc,planta1,EUGL SC,arauco,10,20,directo_planta,N_A,N_A,bosque,cosecha


In [119]:
### Carga de Almacenes ###
base_almacenes = pd.read_excel('../02_data/almacenes.xlsx', sheet_name='almacenes')
no_almacenes = len(base_almacenes)

alm_mes_temp = base_almacenes[['id_almacen', 'zona']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp = pd.merge(alm_mes_temp, periodos, how='left', on='zona')
alm_mes_temp = alm_mes_temp[['id_almacen', 'mes_real', 'mes_planificado', 'temporada']].copy().drop_duplicates(ignore_index=True)
alm_mes_temp

,id_almacen,mes_real,mes_planificado,temporada
0,alm1,2023-09,0,0
1,alm1,2023-10,1,1
2,alm1,2023-11,2,1
3,alm1,2023-12,3,1
4,alm1,2024-01,4,1
...,...,...,...,...
255,alm24,2024-05,8,0
256,alm24,2024-06,9,0
257,alm24,2024-07,10,0
258,alm24,2024-08,11,0


In [120]:
#Stock almacen EN MES 0
base_stock_almacenes = pd.read_excel('../02_data/almacenes_stock.xlsx', sheet_name='cierrestock')
work_alm = pd.merge(base_almacenes, base_stock_almacenes, how='left', on='id_almacen')
work_alm['ant_meses'] = work_alm['ant_meses'].astype(np.int64).astype(str)
work_alm = work_alm.rename(columns={'ant_meses':'antiguedad'})
work_alm = pd.merge(work_alm, rangos_demanda_ext, on='antiguedad', how='left')
work_alm['mes_planificado'] = 1

In [128]:
#Modifica demanda del  primer periodo de acuerdo a disponibilidad.
lista_periodos = list(periodos['mes_planificado'].sort_values(ascending=True).unique())
dpla = demanda_plantas.copy()

def algo1(mes, planta, producto, dpla):
    '''
    Modifica la demanda de acuerdo a la disponibilidad de antiguo.
    '''
    dpla = dpla.sort_values(by='inicio_rango', ascending=False).reset_index(drop=True)

    #Si la planta no tiene demanda en el mes actual, se continua el ciclo.
    if len(dpla) == 0:
        print(f'No existen demandas a cumplir en el periodo {mes}')
        return dpla

    #En una (planta, producto, mes) solo puede haber un "id_tipo_rango".
    tipo_rango_mes = dpla['id_tipo_rango'].unique()[0]

    #Se verifica que exista en los almacenes la cantidad de producto disponible por rango-antiguedad y con disponibilidad en la temporada.
    new_alm = work_alm.copy()
    new_alm = new_alm[(new_alm['nombre_planta'] == planta) 
                        & (new_alm['producto'] == producto)
                        & (new_alm['mes_planificado'] == mes)
                        & (new_alm['id_tipo_rango'] == tipo_rango_mes)]
    
    #Temporada del almacen
    temp_alm = alm_mes_temp.copy()
    temp_alm = temp_alm[(temp_alm['mes_planificado'] == mes)]

    new_alm = pd.merge(new_alm, temp_alm, how='left', on='id_almacen')
    inv_drops = new_alm.loc[(new_alm['temporada'] == 0) & (new_alm['carpeta'] == 'tierra')].index
    new_alm = new_alm.drop(inv_drops, axis=0)
    new_alm = new_alm.groupby(['nombre_planta', 'producto', 'rango'], as_index=False).agg(volumen=('cierre_stock', 'sum'))
    new_alm = new_alm.sort_values(by='rango', ascending=False).reset_index(drop=True)
    # display(new_alm)

    dpla = pd.merge(dpla, new_alm, how='left', on=['nombre_planta', 'producto', 'rango'])
    dpla['volumen'] = dpla['volumen'].fillna(0)

    #Algoritmo Ale
    for ix in dpla.index[:-1]:
        if dpla.at[ix, 'volumen'] < dpla.at[ix, 'demanda_rango']:
            dpla.at[ix + 1, 'demanda_rango'] +=  dpla.at[ix, 'demanda_rango'] - dpla.at[ix, 'volumen']
            dpla.at[ix, 'demanda_rango'] = dpla.at[ix, 'volumen']
    dpla = dpla.drop(columns=['volumen']).sort_values(by=['mes_planificado', 'inicio_rango'])

    return dpla



def algo2(mes, planta, producto, dpla):
    dpla = dpla.sort_values(by='inicio_rango', ascending=True).reset_index(drop=True)

    #Si la planta no tiene demanda en el mes actual, se continua el ciclo.
    if len(dpla) == 0:
        print(f'No existen demandas a cumplir en el periodo {mes}')
        return dpla, 0
    
    vol_demanda = dpla['demanda_rango'].sum()
    
    #Se filtran los ingresos de planta directo.
    auxx = vol_ingresos.copy()
    auxx = auxx.loc[(auxx['nombre_planta'] == planta) 
                    & (auxx['producto'] == producto) 
                    & (auxx['mes_planificado'] == mes)
                    & (auxx['tipo_ingreso'] == 'directo_planta')]
    auxx = auxx.groupby(['nombre_planta', 'producto', 'mes_planificado'], as_index=False).agg({'volumen':'sum'})

    #Indicadores
    vol_fresco = auxx.at[0, 'volumen']
    vol_stock = vol_fresco - vol_demanda
    if vol_stock < 0:
        vol_stock = 0.0

    print(f'vol_demanda: {vol_demanda}, vol_ingreso_directo: {vol_fresco}, vol_stock: {vol_stock}')

    #Algoritmo Ale
    if vol_fresco > aux1.at[0, 'demanda_rango']:
        dpla.at[1, 'demanda_rango'] = dpla.at[1, 'demanda_rango'] + dpla.at[0, 'demanda_rango'] - vol_fresco
        dpla.at[0, 'demanda_rango'] = vol_fresco

    for ix in dpla.index[1:-1]:
        if dpla.at[ix, 'demanda_rango'] < 0:
            dpla.at[ix+1, 'demanda_rango'] = dpla.at[ix+1, 'demanda_rango'] + dpla.at[ix, 'demanda_rango']
            dpla.at[ix, 'demanda_rango'] = 0

    demanda_faltante = dpla.at[0, 'demanda_rango'] - vol_fresco
    dpla = dpla.sort_values(by=['mes_planificado', 'inicio_rango'])
    return dpla, demanda_faltante

def do_tabla_venta_ingresos(mes, planta, producto, tabla_ventas_ingresos):
    #Se buscan los volumenes de ingresos directos
    aux1 = vol_ingresos.copy()
    aux1 = aux1.loc[(aux1['mes_planificado'] == mes)
                    & (aux1['nombre_planta'] == planta)
                    & (aux1['producto'] == producto)
                    & (aux1['tipo_ingreso'] == 'directo_planta')]

    #Se traspasan a la tabla de ventas
    tvi_temporal = pd.DataFrame()
    for ix in aux1.index:
        aux_tvi = pd.DataFrame([[aux1.at[ix, 'id_ingreso'], mes, aux1.at[ix, 'volumen'], 0]], columns=['id_ingreso', 'mes_planificado', 'vol_venta', 'vol_almacen'] )
        tvi_temporal = pd.concat([tvi_temporal, aux_tvi], ignore_index=True)
    return pd.concat([tabla_ventas_ingresos, tvi_temporal], ignore_index=True)


def algo4(mes, planta, producto, demanda_faltante, tabla_ventas_ingresos):
    #Se filtran los ingresos de planta directo.
    aux1 = vol_ingresos.copy()
    aux1 = aux1.loc[(aux1['nombre_planta'] == planta) 
                    & (aux1['producto'] == producto) 
                    & (aux1['mes_planificado'] == mes)
                    & (aux1['tipo_ingreso'] == 'libre')]
    
    tvi_temporal = pd.DataFrame()
    for ix in aux1.index:
        if demanda_faltante >= aux1.at[ix, 'volumen']:
            aux_tvi = pd.DataFrame([[aux1.at[ix, 'id_ingreso'], mes, aux1.at[ix, 'volumen'], 0]], columns=['id_ingreso', 'mes_planificado', 'vol_venta', 'vol_almacen'] )
            tvi_temporal = pd.concat([tvi_temporal, aux_tvi], ignore_index=True)
            demanda_faltante = demanda_faltante - aux1.at[ix, 'volumen']

        else:
            aux_tvi = pd.DataFrame([[aux1.at[ix, 'id_ingreso'], mes, demanda_faltante, aux1.at[ix, 'volumen'] - demanda_faltante]], columns=['id_ingreso', 'mes_planificado', 'vol_venta', 'vol_almacen'] )
            tvi_temporal = pd.concat([tvi_temporal, aux_tvi], ignore_index=True)
            demanda_faltante = 0
        
        tabla_ventas_ingresos = pd.concat([tabla_ventas_ingresos, tvi_temporal], ignore_index=True)

    return demanda_faltante, tabla_ventas_ingresos


def algo5(mes, planta, producto, tabla_ventas_ingresos, demanda_faltante):
    return
    




In [132]:
mes=1
planta='planta1'
producto='EUGL CC'

dpla = dpla.sort_values(by='inicio_rango', ascending=True).reset_index(drop=True)

aux_alm = work_alm.copy()
aux_alm['antiguedad'] = aux_alm['antiguedad'].astype(np.int64)
aux_alm = aux_alm.loc[(aux_alm['nombre_planta'] == planta) 
                & (aux_alm['producto'] == producto) 
                & (aux_alm['mes_planificado'] == mes)].sort_values(by=['prioridad', 'antiguedad'], ascending=True)

temporal_tva = aux_alm[['id_almacen']].drop_duplicates(ignore_index=True)
temporal_tva


0

In [129]:
tabla_ventas_ingresos = pd.DataFrame()
tabla_ventas_almacenes = pd.DataFrame()

dpla = demanda_plantas.copy()
for mes in [1]:
    for planta in df['nombre_planta'].unique():
        for producto in df[df['nombre_planta'] == planta]['producto'].unique():
            print(planta, producto, mes)
            dpla = demanda_plantas.copy()
            dpla = dpla[(dpla['nombre_planta'] == planta) & (dpla['producto'] == producto) & (dpla['mes_planificado'] == mes)].copy()

            dpla = algo1(mes, planta, producto, dpla)
            dpla, demanda_faltante = algo2(mes, planta, producto, dpla)
            display(dpla)
            print(demanda_faltante)
            tabla_ventas_ingresos = do_tabla_venta_ingresos(mes, planta, producto, tabla_ventas_ingresos)
            demanda_faltante, tabla_ventas_ingresos = algo4(mes, planta, producto, demanda_faltante, tabla_ventas_ingresos)
            print(demanda_faltante)

planta1 EUGL CC 1
vol_demanda: 82.18999999999998, vol_ingreso_directo: 30, vol_stock: 0.0


,nombre_planta,producto,mes_planificado,id_tipo_rango,rango,inicio_rango,fin_rango,demanda_rango
0,planta1,EUGL CC,1,1,0-3,0,3,58.971
1,planta1,EUGL CC,1,1,4-6,4,6,15.000
2,planta1,EUGL CC,1,1,7-9,7,9,8.219


28.97099999999999
0


In [130]:
tabla_ventas_ingresos

,id_ingreso,mes_planificado,vol_venta,vol_almacen
0,10,1,15.000,0.000
1,11,1,5.000,0.000
2,12,1,5.000,0.000
3,16,1,5.000,0.000
4,28,1,28.971,1.029
